# Example: using a different force field

Here we show how to use MDPOW to calculate partition coefficients using an force field that isn't included in the package. To follow along, install `jupyter` in your `mdpow` environment.

To implement a new force field, you will need:

* `ITP` files for the molecule, the solvents, ions and also the general atom type definitions file (usually just named after the force field itself).
* `MDP` files for the energy minimisation, initial relaxation, NPT ensemble run and free energy calculation.
* Structure files (`.gro` or `.pdb`) for the solute and non-aqueous solvent. If you are using a type of water that does not come bundled with GROMACS, like in this example, you will also need to create an equilibrated box of pure water.

The first thing we'll do is to download the files we need for Martini 3.0.

In [1]:
from pathlib import Path
from typing import Optional

import requests as req
from zipfile import ZipFile


HERE = Path(".")
MARTINI_FF = HERE / "martini.ff"

MARTINI_ITP = MARTINI_FF / "forcefield.itp"
MARTINI_IONS = MARTINI_FF / "martini_v3.0.0_ions_v1.itp"
MARTINI_SMALL_MOLS = MARTINI_FF / "martini_v3.0.0_small_molecules_v1.itp"
MARTINI_SOLVENTS = MARTINI_FF / "martini_v3.0.0_solvents_v1.itp"
BENZENE_ITP = MARTINI_FF / "BENZ.itp"

MARTINI_WATER = HERE / "water.gro"
MARTINI_OCTANOL = HERE / "octanol.gro"
MARTINI_BENZENE = MARTINI_FF / "output-conect.pdb"

EM_FILE = MARTINI_FF / "em.mdp"
EQ_FILE = MARTINI_FF / "eq.mdp"
RUN_FILE = MARTINI_FF / "run.mdp"


def download_file(
    url: str, out: Optional[Path] = None, chunk_size: int = 128, overwrite: bool = False
):
    """Utility function to download files."""
    if out is None:
        out = HERE / Path(url).name

    if out.exists() and not overwrite:
        return

    r = req.get(url, stream=True)
    r.raise_for_status()

    with out.open("wb") as f:
        for chunk in r.iter_content(chunk_size=chunk_size):
            f.write(chunk)


ZIP_DOWNLOAD = {
    HERE
    / "BENZ.itp": "https://mad.ibcp.fr/api/molecule/download?id=731653322400583591&filename=BENZ.zip"
}
DOWNLOADS = {
    MARTINI_ITP: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0.itp",
    MARTINI_IONS: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_ions_v1.itp",
    MARTINI_SMALL_MOLS: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_small_molecules_v1.itp",
    MARTINI_SOLVENTS: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_solvents_v1.itp",
}
DOWNLOADS.update(ZIP_DOWNLOAD)

for fname, url in DOWNLOADS.items():
    download_file(url, fname)

for zip_file in ZIP_DOWNLOAD.keys():
    with ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall(MARTINI_FF)

This should have downloaded several files to your workspace.

We also need to make a `watermodels.dat` file in the `martini.ff` subdirectory.


In [2]:
WATERMODEL_DAT = MARTINI_FF / "watermodels.dat"

WATERMODEL_DAT.write_text("martini-water\tMARTINI-WATER\tMartini default water model.")

56

Next, we set up the files for the Martini 3.0 forcefield.

In [11]:
from mdpow.forcefields import Forcefield, GromacsSolventModel

MARTINI = Forcefield(
    "Martini",
    solvent_models={
        "water": GromacsSolventModel(
            identifier="martini-water",
            itp=MARTINI_SOLVENTS.absolute(),
            coordinates=MARTINI_WATER.absolute(),
            forcefield="Martini",
        ),
        "octanol": GromacsSolventModel(
            identifier="octanol",
            itp=MARTINI_SOLVENTS.absolute(),
            coordinates=MARTINI_OCTANOL.absolute(),
            forcefield="Martini",
        ),
    },
    forcefield_dir=MARTINI_FF.absolute(),
    ions_itp=MARTINI_IONS.absolute(),
    default_water_itp=MARTINI_SOLVENTS.absolute(),
    default_water_model="martini-water",
)

In [12]:
from dataclasses import asdict

for solvent_name, solvent_model in MARTINI.solvent_models.items():
    print(solvent_name)
    print(solvent_model)

water
<MARTINI-WATER water: identifier=martini-water, ff=Martini>
octanol
<OCTANOL water: identifier=octanol, ff=Martini>


In [6]:
from mdpow.equil import WaterSimulation

sim = WaterSimulation(molecule="BENZ", ff_class=MARTINI)
sim.topology(str(BENZENE_ITP))
print(sim.solvent)
# sim.solvate(struct=MARTINI_BENZENE)
# sim.energy_minimize()
# sim.MD_relaxed(runtime=5)  # should be at least 1e3 ps for production not just 5 ps

mdpow.equil : INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top] Created topology 'system.top' that includes 'BENZ.itp'


{'itp': 'martini-water.itp', 'box': 'martini-water.gro', 'distance': 1.0}


In [ ]:

# run simulation externally or use MDrunner
# (see docs for using mpi etc)
import gromacs

r = gromacs.run.MDrunner(
    dirname=sim.dirs["MD_relaxed"],
    deffnm="md",
    c="md.pdb",
    cpi=True,
    append=True,
    v=True,
)
r.run()  # runs mdrun in the python shell


sim.MD(
    runtime=10, qscript=["local.sh"]
)  # should be at least 10e3 ps for production, not just 10 ps
# run simulation
r = gromacs.run.MDrunner(
    dirname=sim.dirs["MD_NPT"], deffnm="md", c="md.pdb", cpi=True, append=True, v=True
)
r.run()  # runs mdrun in the python shell


import mdpow.fep

gwat = mdpow.fep.Ghyd(simulation=sim, runtime=10)
gwat.setup()

# run multiple simulations on cluster


O = mdpow.equil.OctanolSimulation(molecule="BNZ")
O.topology("benzene.itp")
O.solvate(struct="benzene.pdb")
O.energy_minimize()
O.MD_relaxed(runtime=0.5)